In [1]:
import os


In [2]:
%pwd

'e:\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'e:\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\Conda\envs\nncls\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\Conda\envs\nncls\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\Conda\envs\nncls\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\Conda\envs\nncls\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\Conda\envs\nncls\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\Conda\envs\nncls\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\Conda\envs\nncls\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\Conda\envs\nncls\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\Conda\envs\nncls\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\Conda\envs\nncls\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\Conda\envs\nncls\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\Conda\envs\nncls\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\Conda\envs\nncls\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\Conda\envs\nncls\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\Conda\envs\nncls\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\Conda\envs\nncls\lib\site-packages\ipykernel\kernelapp

AttributeError: _ARRAY_API not found

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # 1. Reconstruct Architecture
        base_model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=None, 
            include_top=False
        )
        
        # 2. Add custom layers
        x = tf.keras.layers.Flatten()(base_model.output)
        
        # CHANGE THIS LINE: 
        # Your saved model was trained on 2 classes, so this MUST be 2.
        classes = 2 
        
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(x)
        
        # 3. Assemble
        self.model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction)
        
        # 4. Load weights
        # This will now work because the shapes match (25088, 2) == (25088, 2)
        self.model.load_weights(self.config.updated_base_model_path)
        print(f"Success! Model loaded with {classes} classes.")
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Convert Path object to string for Keras compatibility
        model.save(str(path))

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), # Add your preferred optimizer
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e 

[2026-01-20 20:52:25,668: INFO: common: yaml file: E:\Kidney-Disease-Classification-Deep-Learning-Project\config\config.yaml loaded successfully]
[2026-01-20 20:52:25,686: INFO: common: yaml file: E:\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml loaded successfully]
[2026-01-20 20:52:25,688: INFO: common: created directory at: artifacts]
[2026-01-20 20:52:25,690: INFO: common: created directory at: artifacts\training]
Success! Model loaded with 2 classes.
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 138s 6s/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 2/200
 1/23 ━━━━━━━━━━━━━━━━━━━━ 2:10 6s/step - accuracy: 0.5000 - loss: nan

e:\Conda\envs\nncls\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 306ms/step - accuracy: 0.5000 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 0.5140 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 300ms/step - accuracy: 0.5625 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 131s 6s/step - accuracy: 0.5169 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 6/200
 1/23 ━━━━━━━━━━━━━━━━━━━━ 1:57 5s/step - accuracy: 0.5000 - loss: nan

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\data_ingestion\\kidney-ct-scan-image\\Normal\\Normal- (637).jpg'
Traceback (most recent call last):

  File "C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)

  File "C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "e:\Conda\envs\nncls\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py", line 264, in _finite_generator
    yield self._standardize_batch(self.py_